In [1]:

import pandas as pd
import numpy as np

# create outputs for travel model (transcad)

In [2]:
hdf=pd.HDFStore('./data/all_semcog_data_fix.h5')

parcels=hdf.get('parcels')
buildings=hdf.get('buildings')
hh=hdf.get('households')
persons=hdf.get('persons')


In [3]:
#hh.dtypes

In [4]:
#hh.building_id=hh.building_id.astype(np.int64)

In [5]:
#join zone_id
#sanitize input tables, in case df1 has joint ids out of range of ids(index values) in df2 
# def syncids(df1,df2,idcolumn):
#     print df1.shape
#     set1=set(df1[idcolumn].unique())
#     set2=set(df2.index.values)
#     missing_pids= [x for x in set1 if x not in set2]
#     print "# of missing ids from df1:    ", len(missing_pids)
#     print 'first 10 missing:',missing_pids[:10]
#     df1=df1[~(df1[idcolumn].isin(missing_pids))]
#     print 'after clean up', df1.shape
#     return df1

In [6]:
#buildings=syncids(buildings,parcels,'parcel_id')

In [7]:
#syncids(hh,buildings,'building_id')

In [8]:
#persons=syncids(persons,hh,'household_id')

In [3]:
buildings=pd.merge(buildings,parcels[['zone_id']], left_on='parcel_id',right_index='True', how='left')
hh=pd.merge(hh,buildings[['zone_id']], left_on='building_id',right_index='True', how='left')
persons=pd.merge(persons,hh[['zone_id']], left_on='household_id',right_index='True', how='left')

In [4]:
print buildings.zone_id.dtype
print hh.zone_id.dtype
print persons.zone_id.dtype

float64
float64
float64


In [5]:

#jobs = jobs.to_frame()

zonal_indicators = pd.DataFrame(index=np.unique(parcels.zone_id.values))
zonal_indicators.index.name='zone_id'

zonal_indicators['Households'] = hh.groupby('zone_id').size()
zonal_indicators['HHPop'] = hh.groupby('zone_id').persons.sum()
#zonal_indicators['EmpPrinc'] = jobs.groupby('zone_id').size()
zonal_indicators['workers'] = hh.groupby('zone_id').workers.sum()
zonal_indicators['Agegrp1'] = persons[persons.age<=4].groupby('zone_id').size() #???
zonal_indicators['Agegrp2'] = persons[(persons.age>=5) & (persons.age<=17)].groupby('zone_id').size() #???
zonal_indicators['Agegrp3'] = persons[(persons.age>=18) & (persons.age<=34)].groupby('zone_id').size() #???
#zonal_indicators['Age_18to34'] = persons[(persons.age>=18) & (persons.age<=34)].groupby('zone_id').size()
zonal_indicators['Agegrp4'] = persons[(persons.age>=35) & (persons.age<=64)].groupby('zone_id').size() #???
zonal_indicators['Agegrp5'] = persons[persons.age>=65].groupby('zone_id').size() #???
#enroll_ratios = pd.read_csv("data/schdic_taz10.csv")
#school_age_by_district = pd.DataFrame({'children':persons[(persons.age>=5) & (persons.age<=17)].groupby('school_district_id').size()})
#enroll_ratios = pd.merge(enroll_ratios,school_age_by_district,left_on='school_district_id',right_index=True)
#enroll_ratios['enrolled'] = enroll_ratios.enroll_ratio*enroll_ratios.children
#enrolled = enroll_ratios.groupby('zone_id').enrolled.sum()
#zonal_indicators['K12Enroll'] = np.round(enrolled)
#zonal_indicators['PopDens'] = zonal_indicators.HHPop/(parcels.groupby('zone_id').parcel_sqft.sum()/43560)
#zonal_indicators['EmpDens'] = zonal_indicators.EmpPrinc/(parcels.groupby('zone_id').parcel_sqft.sum()/43560)
# zonal_indicators['EmpBasic'] = jobs[jobs.sector_id.isin([1,3])].groupby('zone_id').size()
# zonal_indicators['EmpNonBas'] = jobs[~jobs.sector_id.isin([1,3])].groupby('zone_id').size()
#zonal_indicators['Natural_Resource_and_Mining'] = jobs[jobs.sector_id==1].groupby('zone_id').size()
#zonal_indicators['sector2'] = jobs[jobs.sector_id==2].groupby('zone_id').size()
#zonal_indicators['Manufacturing'] = jobs[jobs.sector_id==3].groupby('zone_id').size()
#zonal_indicators['Wholesale_Trade'] = jobs[jobs.sector_id==4].groupby('zone_id').size()
#zonal_indicators['Retail_Trade'] = jobs[jobs.sector_id==5].groupby('zone_id').size()
#zonal_indicators['Transportation_and_Warehousing'] = jobs[jobs.sector_id==6].groupby('zone_id').size()
#zonal_indicators['Utilities'] = jobs[jobs.sector_id==7].groupby('zone_id').size()
#zonal_indicators['Information'] = jobs[jobs.sector_id==8].groupby('zone_id').size()
#zonal_indicators['Financial_Service'] = jobs[jobs.sector_id==9].groupby('zone_id').size()
#zonal_indicators['Professional_Science_Tec'] = jobs[jobs.sector_id==10].groupby('zone_id').size()
#zonal_indicators['Management_of_CompEnt'] = jobs[jobs.sector_id==11].groupby('zone_id').size()
#zonal_indicators['Administrative_Support_and_WM'] = jobs[jobs.sector_id==12].groupby('zone_id').size()
#zonal_indicators['Education_Services'] = jobs[jobs.sector_id==13].groupby('zone_id').size()
## zonal_indicators['sector14'] = jobs[jobs.sector_id==14].groupby('zone_id').size()
## zonal_indicators['sector15'] = jobs[jobs.sector_id==15].groupby('zone_id').size()
#zonal_indicators['Health_Care_and_SocialSer'] = jobs[np.in1d(jobs.sector_id,[14,15,19])].groupby('zone_id').size()
#zonal_indicators['Leisure_and_Hospitality'] = jobs[jobs.sector_id==16].groupby('zone_id').size()
#zonal_indicators['Other_Services'] = jobs[jobs.sector_id==17].groupby('zone_id').size()
#zonal_indicators['sector18'] = jobs[jobs.sector_id==18].groupby('zone_id').size()
#zonal_indicators['sector19'] = jobs[jobs.sector_id==19].groupby('zone_id').size()
#zonal_indicators['Public_Administration'] = jobs[jobs.sector_id==20].groupby('zone_id').size()

#hh['schoolkids'] = persons[(persons.age>=5) & (persons.age<=17)].groupby('household_id').size()
#hh.schoolkids = hh.schoolkids.fillna(0)
hh['Age_5to17'] = persons[(persons.age>=5) & (persons.age<=17)].groupby('household_id').size()
hh.Age_5to17 = hh.Age_5to17.fillna(0)
hh['Age_75'] = persons[persons.age>=75].groupby('household_id').size()
hh.Age_75 = hh.Age_5to17.fillna(0)
#zonal_indicators['PrCh21'] = hh[(hh.persons==2) & (hh.schoolkids==1)].groupby('zone_id').size()
#zonal_indicators['PrCh31'] = hh[(hh.persons==3) & (hh.schoolkids==1)].groupby('zone_id').size()
#zonal_indicators['PrCh32'] = hh[(hh.persons==3) & (hh.schoolkids==2)].groupby('zone_id').size()
#zonal_indicators['PrCh41'] = hh[(hh.persons==4) & (hh.schoolkids==1)].groupby('zone_id').size()
#zonal_indicators['PrCh42'] = hh[(hh.persons==4) & (hh.schoolkids==2)].groupby('zone_id').size()
#zonal_indicators['PrCh43'] = hh[(hh.persons==4) & (hh.schoolkids>=3)].groupby('zone_id').size()
#zonal_indicators['PrCh51'] = hh[(hh.persons>=5) & (hh.schoolkids==1)].groupby('zone_id').size()
#zonal_indicators['PrCh52'] = hh[(hh.persons>=5) & (hh.schoolkids==2)].groupby('zone_id').size()
#zonal_indicators['PrCh53'] = hh[(hh.persons>=5) & (hh.schoolkids>=3)].groupby('zone_id').size()
hh['quartile'] = pd.Series(pd.qcut(hh.income,4,labels=False), index=hh.index)+1

#zonal_indicators['Inc1HHsze1'] = hh[(hh.persons==1) & (hh.quartile==1)].groupby('zone_id').size()
#zonal_indicators['Inc2HHsze1'] = hh[(hh.persons==1) & (hh.quartile==2)].groupby('zone_id').size()
#zonal_indicators['Inc3HHsze1'] = hh[(hh.persons==1) & (hh.quartile==3)].groupby('zone_id').size()
#zonal_indicators['Inc4HHsze1'] = hh[(hh.persons==1) & (hh.quartile==4)].groupby('zone_id').size()
#zonal_indicators['Inc1HHsze2'] = hh[(hh.persons==2) & (hh.quartile==1)].groupby('zone_id').size()
#zonal_indicators['Inc2HHsze2'] = hh[(hh.persons==2) & (hh.quartile==2)].groupby('zone_id').size()
#zonal_indicators['Inc3HHsze2'] = hh[(hh.persons==2) & (hh.quartile==3)].groupby('zone_id').size()
#zonal_indicators['Inc4HHsze2'] = hh[(hh.persons==2) & (hh.quartile==4)].groupby('zone_id').size()
#zonal_indicators['Inc1HHsze3'] = hh[(hh.persons==3) & (hh.quartile==1)].groupby('zone_id').size()
#zonal_indicators['Inc2HHsze3'] = hh[(hh.persons==3) & (hh.quartile==2)].groupby('zone_id').size()
#zonal_indicators['Inc3HHsze3'] = hh[(hh.persons==3) & (hh.quartile==3)].groupby('zone_id').size()
#zonal_indicators['Inc4HHsze3'] = hh[(hh.persons==3) & (hh.quartile==4)].groupby('zone_id').size()
#zonal_indicators['Inc1HHsze4'] = hh[(hh.persons==4) & (hh.quartile==1)].groupby('zone_id').size()
#zonal_indicators['Inc2HHsze4'] = hh[(hh.persons==4) & (hh.quartile==2)].groupby('zone_id').size()
#zonal_indicators['Inc3HHsze4'] = hh[(hh.persons==4) & (hh.quartile==3)].groupby('zone_id').size()
#zonal_indicators['Inc4HHsze4'] = hh[(hh.persons==4) & (hh.quartile==4)].groupby('zone_id').size()
#zonal_indicators['Inc1HHsze5p'] = hh[(hh.persons>=5) & (hh.quartile==1)].groupby('zone_id').size()
#zonal_indicators['Inc2HHsze5p'] = hh[(hh.persons>=5) & (hh.quartile==2)].groupby('zone_id').size()
#zonal_indicators['Inc3HHsze5p'] = hh[(hh.persons>=5) & (hh.quartile==3)].groupby('zone_id').size()
#zonal_indicators['Inc4HHsze5p'] = hh[(hh.persons>=5) & (hh.quartile==4)].groupby('zone_id').size()
#zonal_indicators['WkAu10'] = hh[(hh.workers==1) & (hh.cars==0)].groupby('zone_id').size()
#zonal_indicators['WkAu11'] = hh[(hh.workers==1) & (hh.cars==1)].groupby('zone_id').size()
#zonal_indicators['WkAu12'] = hh[(hh.workers==1) & (hh.cars==2)].groupby('zone_id').size()
#zonal_indicators['WkAu13'] = hh[(hh.workers==1) & (hh.cars>=3)].groupby('zone_id').size()
#zonal_indicators['WkAu20'] = hh[(hh.workers==2) & (hh.cars==0)].groupby('zone_id').size()
#zonal_indicators['WkAu21'] = hh[(hh.workers==2) & (hh.cars==1)].groupby('zone_id').size()
#zonal_indicators['WkAu22'] = hh[(hh.workers==2) & (hh.cars==2)].groupby('zone_id').size()
#zonal_indicators['WkAu23'] = hh[(hh.workers==2) & (hh.cars>=3)].groupby('zone_id').size()
#zonal_indicators['WkAu30'] = hh[(hh.workers>=3) & (hh.cars==0)].groupby('zone_id').size()
#zonal_indicators['WkAu31'] = hh[(hh.workers>=3) & (hh.cars==1)].groupby('zone_id').size()
#zonal_indicators['WkAu32'] = hh[(hh.workers>=3) & (hh.cars==2)].groupby('zone_id').size()
#zonal_indicators['WkAu33'] = hh[(hh.workers>=3) & (hh.cars>=3)].groupby('zone_id').size()

#zonal_indicators['PrAu10'] = hh[(hh.persons==1) & (hh.cars==0)].groupby('zone_id').size()
#zonal_indicators['PrAu11'] = hh[(hh.persons==1) & (hh.cars==1)].groupby('zone_id').size()
#zonal_indicators['PrAu12'] = hh[(hh.persons==1) & (hh.cars==2)].groupby('zone_id').size()
#zonal_indicators['PrAu13'] = hh[(hh.persons==1) & (hh.cars>=3)].groupby('zone_id').size()
#zonal_indicators['PrAu20'] = hh[(hh.persons==2) & (hh.cars==0)].groupby('zone_id').size()
#zonal_indicators['PrAu21'] = hh[(hh.persons==2) & (hh.cars==1)].groupby('zone_id').size()
#zonal_indicators['PrAu22'] = hh[(hh.persons==2) & (hh.cars==2)].groupby('zone_id').size()
#zonal_indicators['PrAu23'] = hh[(hh.persons==2) & (hh.cars>=3)].groupby('zone_id').size()
#zonal_indicators['PrAu30'] = hh[(hh.persons==3) & (hh.cars==0)].groupby('zone_id').size()
#zonal_indicators['PrAu31'] = hh[(hh.persons==3) & (hh.cars==1)].groupby('zone_id').size()
#zonal_indicators['PrAu32'] = hh[(hh.persons==3) & (hh.cars==2)].groupby('zone_id').size()
#zonal_indicators['PrAu33'] = hh[(hh.persons==3) & (hh.cars>=3)].groupby('zone_id').size()
#zonal_indicators['PrAu40'] = hh[(hh.persons==4) & (hh.cars==0)].groupby('zone_id').size()
#zonal_indicators['PrAu41'] = hh[(hh.persons==4) & (hh.cars==1)].groupby('zone_id').size()
#zonal_indicators['PrAu42'] = hh[(hh.persons==4) & (hh.cars==2)].groupby('zone_id').size()
#zonal_indicators['PrAu43'] = hh[(hh.persons==4) & (hh.cars>=3)].groupby('zone_id').size()
#zonal_indicators['PrAu50'] = hh[(hh.persons>=5) & (hh.cars==0)].groupby('zone_id').size()
#zonal_indicators['PrAu51'] = hh[(hh.persons>=5) & (hh.cars==1)].groupby('zone_id').size()
#zonal_indicators['PrAu52'] = hh[(hh.persons>=5) & (hh.cars==2)].groupby('zone_id').size()
#zonal_indicators['PrAu53'] = hh[(hh.persons>=5) & (hh.cars>=3)].groupby('zone_id').size()
		
#zonal_indicators['inc1nc'] = hh[(hh.quartile==1) & (hh.children==0)].groupby('zone_id').size()
#zonal_indicators['inc1wc'] = hh[(hh.quartile==1) & (hh.children>0)].groupby('zone_id').size()
#zonal_indicators['inc2nc'] = hh[(hh.quartile==2) & (hh.children==0)].groupby('zone_id').size()
#zonal_indicators['inc2wc'] = hh[(hh.quartile==2) & (hh.children>0)].groupby('zone_id').size()
#zonal_indicators['inc3nc'] = hh[(hh.quartile==3) & (hh.children==0)].groupby('zone_id').size()
#zonal_indicators['inc3wc'] = hh[(hh.quartile==3) & (hh.children>0)].groupby('zone_id').size()
#zonal_indicators['inc4nc'] = hh[(hh.quartile==4) & (hh.children==0)].groupby('zone_id').size()
#zonal_indicators['inc4wc'] = hh[(hh.quartile==4) & (hh.children>0)].groupby('zone_id').size()

#zonal_indicators['Inc1w0'] = hh[(hh.quartile==1) & (hh.workers==0)].groupby('zone_id').size()
#zonal_indicators['Inc1w1'] = hh[(hh.quartile==1) & (hh.workers==1)].groupby('zone_id').size()
#zonal_indicators['Inc1w2'] = hh[(hh.quartile==1) & (hh.workers==2)].groupby('zone_id').size()
#zonal_indicators['Inc1w3p'] = hh[(hh.quartile==1) & (hh.workers>=3)].groupby('zone_id').size()
#zonal_indicators['Inc2w0'] = hh[(hh.quartile==2) & (hh.workers==0)].groupby('zone_id').size()
#zonal_indicators['Inc2w1'] = hh[(hh.quartile==2) & (hh.workers==1)].groupby('zone_id').size()
#zonal_indicators['Inc2w2'] = hh[(hh.quartile==2) & (hh.workers==2)].groupby('zone_id').size()
#zonal_indicators['Inc2w3p'] = hh[(hh.quartile==2) & (hh.workers>=3)].groupby('zone_id').size()
#zonal_indicators['Inc3w0'] = hh[(hh.quartile==3) & (hh.workers==0)].groupby('zone_id').size()
#zonal_indicators['Inc3w1'] = hh[(hh.quartile==3) & (hh.workers==1)].groupby('zone_id').size()
#zonal_indicators['Inc3w2'] = hh[(hh.quartile==3) & (hh.workers==2)].groupby('zone_id').size()
#zonal_indicators['Inc3w3p'] = hh[(hh.quartile==3) & (hh.workers>=3)].groupby('zone_id').size()
#zonal_indicators['Inc4w0'] = hh[(hh.quartile==4) & (hh.workers==0)].groupby('zone_id').size()
#zonal_indicators['Inc4w1'] = hh[(hh.quartile==4) & (hh.workers==1)].groupby('zone_id').size()
#zonal_indicators['Inc4w2'] = hh[(hh.quartile==4) & (hh.workers==2)].groupby('zone_id').size()
#zonal_indicators['Inc4w3p'] = hh[(hh.quartile==4) & (hh.workers>=3)].groupby('zone_id').size()

#zonal_indicators['Workers4HH_IncomeGroup1'] = hh[hh.quartile==1].groupby('zone_id').workers.sum()
#zonal_indicators['Workers4HH_IncomeGroup2'] = hh[hh.quartile==2].groupby('zone_id').workers.sum()
#zonal_indicators['Workers4HH_IncomeGroup3'] = hh[hh.quartile==3].groupby('zone_id').workers.sum()
#zonal_indicators['Workers4HH_IncomeGroup4'] = hh[hh.quartile==4].groupby('zone_id').workers.sum()

buildings['residential_sqft']=buildings.sqft_per_unit*buildings.residential_units
zonal_indicators['residential_sqft'] = buildings[buildings.building_type_id.isin([81,82,83,84])].groupby('zone_id').residential_sqft.sum()
zonal_indicators['commercial_sqft'] = buildings[buildings.building_type_id.isin([21,22,25,26, 61, 62])].groupby('zone_id').non_residential_sqft.sum()
zonal_indicators['office_sqft'] = buildings[buildings.building_type_id.isin([23,24])].groupby('zone_id').non_residential_sqft.sum()
zonal_indicators['industrial_sqft'] = buildings[buildings.building_type_id.isin([31,32,33])].groupby('zone_id').non_residential_sqft.sum()
zonal_indicators['medical_sqft'] = buildings[buildings.building_type_id.isin([51,52,53])].groupby('zone_id').non_residential_sqft.sum()
zonal_indicators['institutional_sqft'] = buildings[buildings.building_type_id.isin([11,12,13,14])].groupby('zone_id').non_residential_sqft.sum()


zonal_indicators['AcresEmp1']=parcels[parcels.land_use_type_id.isin(range(1,6)+[8,9,21,22,23])].groupby('zone_id').parcel_sqft.sum()/43560.0
zonal_indicators['AcresEmp2']=parcels[parcels.land_use_type_id.isin(range(1,11)+[15,21,22,23])].groupby('zone_id').parcel_sqft.sum()/43560.0
zonal_indicators['AcresTotal'] = parcels.groupby('zone_id').parcel_sqft.sum()/43560.0

In [13]:
zonal_indicators.head(2)

,Households,HHPop,workers,Agegrp1,Agegrp2,Agegrp3,Agegrp4,Agegrp5,residential_sqft,commercial_sqft,office_sqft,industrial_sqft,medical_sqft,institutional_sqft,AcresEmp1,AcresEmp2,AcresTotal
zone_id,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29340.0,1750.0,821980.0,NaN,NaN,47.577410,47.577410,65.767975
2,600.0,2020.0,488.0,171.0,552.0,498.0,686.0,113.0,757340.0,62694.0,7414.0,108317.0,3033.0,2116.0,9.166001,9.197681,100.971442


In [6]:
enroll = pd.read_csv('schoolEnrollment.csv', usecols=['TDFM_TAZID','ENROLLMENT1516'], index_col='TDFM_TAZID')
enroll.rename(columns={'ENROLLMENT1516':'k12enroll'},inplace=True)
enroll.index.name='zone_id'

In [7]:
enroll.head(2)

,k12enroll
zone_id,
6,0
8,204


In [8]:
gq=pd.read_csv('gq.csv', usecols=['TDFM_TAZID','GQ14'], index_col='TDFM_TAZID')
gq.rename(columns={'GQ14':'Grpop'},inplace=True)
gq.index.name='zone_id'
gq.head(2)

,Grpop
zone_id,
1,0
2,0


In [9]:
emp=pd.read_csv('employment.csv')
emp.head(2)

,TAZCE10_N,sector1,sector2,sector3,sector4,sector5,sector6,sector7,sector8,sector9,sector10,sector11,sector12,sector13,sector14,sector15,sector16,sector17,sector18,EmpPrinc
0,1101,0,5,138,258,44,10,0,0,0,0,0,0,0,0,0,4,28,0,487
1,1102,0,10,39,10,20,17,0,3,18,65,0,85,1,47,0,44,46,0,405


In [10]:
equiv=pd.read_csv('TAZ10_TDFM_equiv.csv',index_col='TDFM_TAZID')
equiv.index.name='zone_id'
equiv=equiv[['TAZCE10_N']]
equiv.head(2)

,TAZCE10_N
zone_id,
2716,70301
2717,70302


In [11]:
zonal2=pd.merge(equiv,zonal_indicators,left_index=True, right_index=True, how='left')

In [12]:
zonal2.head(2)

,TAZCE10_N,Households,HHPop,workers,Agegrp1,Agegrp2,Agegrp3,Agegrp4,Agegrp5,residential_sqft,commercial_sqft,office_sqft,industrial_sqft,medical_sqft,institutional_sqft,AcresEmp1,AcresEmp2,AcresTotal
zone_id,,,,,,,,,,,,,,,,,,
2716,70301,274.0,606.0,291.0,33.0,61.0,173.0,282.0,57.0,392854.0,139621.0,42030.0,7185.0,21801.0,909330.0,104.901354,109.877571,192.176630
2717,70302,862.0,2095.0,1091.0,88.0,355.0,314.0,992.0,346.0,1393343.0,NaN,49768.0,295311.0,NaN,36761.0,41.681864,41.681864,508.694513


In [14]:
zonal2=pd.merge(zonal2, enroll, left_index=True, right_index=True, how='left')

In [15]:
zonal2.head(2)

,TAZCE10_N,Households,HHPop,workers,Agegrp1,Agegrp2,Agegrp3,Agegrp4,Agegrp5,residential_sqft,commercial_sqft,office_sqft,industrial_sqft,medical_sqft,institutional_sqft,AcresEmp1,AcresEmp2,AcresTotal,k12enroll
zone_id,,,,,,,,,,,,,,,,,,,
2716,70301,274.0,606.0,291.0,33.0,61.0,173.0,282.0,57.0,392854.0,139621.0,42030.0,7185.0,21801.0,909330.0,104.901354,109.877571,192.176630,2074.0
2717,70302,862.0,2095.0,1091.0,88.0,355.0,314.0,992.0,346.0,1393343.0,NaN,49768.0,295311.0,NaN,36761.0,41.681864,41.681864,508.694513,NaN


In [16]:
zonal2=pd.merge(zonal2, gq, left_index=True, right_index=True, how='left')
zonal2.head(2)

,TAZCE10_N,Households,HHPop,workers,Agegrp1,Agegrp2,Agegrp3,Agegrp4,Agegrp5,residential_sqft,commercial_sqft,office_sqft,industrial_sqft,medical_sqft,institutional_sqft,AcresEmp1,AcresEmp2,AcresTotal,k12enroll,Grpop
zone_id,,,,,,,,,,,,,,,,,,,,
2716,70301,274.0,606.0,291.0,33.0,61.0,173.0,282.0,57.0,392854.0,139621.0,42030.0,7185.0,21801.0,909330.0,104.901354,109.877571,192.176630,2074.0,0
2717,70302,862.0,2095.0,1091.0,88.0,355.0,314.0,992.0,346.0,1393343.0,NaN,49768.0,295311.0,NaN,36761.0,41.681864,41.681864,508.694513,NaN,1


In [17]:
zonal2.reset_index(inplace=True)

In [18]:
zonal2=pd.merge(zonal2, emp, left_on='TAZCE10_N', right_on='TAZCE10_N', how='left')

In [19]:
zonal2.head(2)

,zone_id,TAZCE10_N,Households,HHPop,workers,Agegrp1,Agegrp2,Agegrp3,Agegrp4,Agegrp5,...,sector10,sector11,sector12,sector13,sector14,sector15,sector16,sector17,sector18,EmpPrinc
0,2716,70301,274.0,606.0,291.0,33.0,61.0,173.0,282.0,57.0,...,96,0,31,242,127,0,133,129,27,1203
1,2717,70302,862.0,2095.0,1091.0,88.0,355.0,314.0,992.0,346.0,...,138,0,81,16,18,0,9,55,0,656


In [20]:
zonal2.drop(['TAZCE10_N',],axis=1, inplace=True)

In [21]:
zonal2.columns.values

array(['zone_id', 'Households', 'HHPop', 'workers', 'Agegrp1', 'Agegrp2',
       'Agegrp3', 'Agegrp4', 'Agegrp5', 'residential_sqft',
       'commercial_sqft', 'office_sqft', 'industrial_sqft', 'medical_sqft',
       'institutional_sqft', 'AcresEmp1', 'AcresEmp2', 'AcresTotal',
       'k12enroll', 'Grpop', 'sector1', 'sector2', 'sector3', 'sector4',
       'sector5', 'sector6', 'sector7', 'sector8', 'sector9', 'sector10',
       'sector11', 'sector12', 'sector13', 'sector14', 'sector15',
       'sector16', 'sector17', 'sector18', 'EmpPrinc'], dtype=object)

In [22]:
zonal2['TotalPop']=zonal2.HHPop + zonal2.Grpop

In [23]:
zonal2.fillna(0,inplace=True)
zonal2.head()

,zone_id,Households,HHPop,workers,Agegrp1,Agegrp2,Agegrp3,Agegrp4,Agegrp5,residential_sqft,...,sector11,sector12,sector13,sector14,sector15,sector16,sector17,sector18,EmpPrinc,TotalPop
0,2716,274.0,606.0,291.0,33.0,61.0,173.0,282.0,57.0,392854.0,...,0,31,242,127,0,133,129,27,1203,606.0
1,2717,862.0,2095.0,1091.0,88.0,355.0,314.0,992.0,346.0,1393343.0,...,0,81,16,18,0,9,55,0,656,2096.0
2,2720,520.0,908.0,527.0,36.0,81.0,238.0,388.0,165.0,632056.0,...,0,28,140,258,0,60,112,0,1343,930.0
3,2722,227.0,401.0,259.0,11.0,26.0,74.0,146.0,144.0,271462.0,...,0,94,113,288,0,843,127,135,3677,439.0
4,2723,392.0,1059.0,502.0,63.0,154.0,193.0,460.0,189.0,767739.0,...,0,32,0,3,0,1,16,0,187,1059.0


In [24]:
hh.to_csv('households.csv')
persons.to_csv('persons.csv')
zonal2.to_csv('transcad_taz.csv')

# basic output tables 

In [ ]:
#nodes
nodes = orca.get_table('nodes').to_frame()
nodes.columns

In [ ]:
nodes.columns

In [ ]:
#nodes[['new_far','sum_total_sqft','sum_parcel_size']]

In [ ]:
#feasibility
feasi=orca.get_table('feasibility')
#feasi.columns

In [ ]:
#parcels
parcels = orca.get_table('parcels').to_frame()
parcels.columns

In [ ]:
parcels.land_use_type_id.unique()

In [ ]:
orca.list_tables()

In [ ]:
#buildings
buildings = orca.get_table('buildings').to_frame()
buildings.columns

In [ ]:
#new buildings
newbldgs=pd.read_csv("new_buildings.csv")
#newbldgs=buildings[buildings.year_built==2016]
#newbldgs

In [ ]:
# zoning
zoning = orca.get_table('zoning').to_frame()
zoning.columns

In [ ]:
lone_house = orca.get_injectable('lone_house').to_frame()
#lone_house

In [ ]:
pcllone=pd.merge(parcels, lone_house, left_index=True, right_index=True, how='left')

In [ ]:
pcllone.to_csv("parcel_lone_house.csv")

# Join network based indicators to ...

In [ ]:
# to parcels
pclnode=pd.merge(parcels, nodes, left_on="_node_id", right_index=True, how='left')
#pclnode.to_csv('parcel_nodes.csv')

In [ ]:
pclnode.to_csv('parcel_nodes_lu_far.csv')

In [ ]:
# to buildings
bldnode=pd.merge(buildings, pclnode, left_on='parcel_id', right_index=True, how='left')

In [ ]:
# to new buildings
newbnode=pd.merge(newbldgs, pclnode, left_on='parcel_id', right_index=True, how='left')

In [ ]:
pclnode['far1']=pclnode.total_sqft/pclnode.parcel_size

In [ ]:
pclnode.columns

In [ ]:
pclnode[['new_far','max_far','parcel_size','total_sqft']].head()

In [ ]:
pclnode[pclnode['new_far']>pclnode['max_far']]

In [ ]:
pclnode[pclnode['new_far']<(pclnode['far1']-0.01)][['new_far','far1','_node_id']]

In [ ]:
pclnode[pclnode['new_parcel_size']<pclnode['parcel_size']][['new_parcel_size','parcel_size']]

In [ ]:
pclnode[(pclnode['new_pct_undev'])<pclnode['pct_undev']][['new_pct_undev','pct_undev']]

# analysis

In [ ]:
#recent buildings correlates to 
bldrecent=bldnode[(bldnode.year_built>2011) & (bldnode.sqft_per_unit>0)]
bldrecent.shape
cort=bldrecent.corr()

In [ ]:
# sqft_per_unit
#cort['sqft_per_unit']
#bldrecent.plot(x='sqft_per_unit', y='ave_unit_sqft')

In [ ]:
# select parcels
parcels.loc[1012389]

In [ ]:
# office use values counts
npp.office.value_counts(sort=True,bins=50)

In [ ]:
#histogram for office prices based on network module
npp=nodes_prices[nodes_prices.medical>0]
npp.medical1k.plot(kind='hist',bins=200)

In [ ]:
#histogram for office prices based on network module
npp=nodes_prices[nodes_prices.office1k>0]
npp.office1k.plot(kind='hist',bins=200)

In [ ]:
pp=parcels[parcels.land_use_type_id==11  ]
pp.parcel_sqft.plot(kind='hist',bins=40, range=[0, 100000])
pp.parcel_sqft.value_counts(sort=True,bins=50)

In [ ]:
pd.unique(parcels.land_use_type_id)

In [ ]:
ps=buildings[(buildings.building_type_id == 81) & (buildings.residential_units == 1)].groupby(by="parcel_id").building_type_id.count() == 1

In [ ]:
parcels.head()

# fix hdf5 data

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
hf=pd.HDFStore('./data/alderaan_semcog_data_fix.h5')

In [ ]:
hf

In [ ]:
hf["buildings"] = hf["buildings"][hf["buildings"]["building_type_id"].notnull()]
hf["buildings"] = hf["buildings"][hf["buildings"]["improvement_value"].notnull()]

In [ ]:
lacal = hf["buildings"]
for c in lacal.columns:
    print c, lacal[c].dtype
    if lacal[c].dtype == np.float64:
        lacal[c] = lacal[c].astype(np.int32)
del hf["buildings"]
hf["buildings"] = lacal

In [ ]:
hf['/zoning']['future_use'].unique()

In [ ]:
for item in hf.keys():
    print item, hf[item].shape

In [ ]:
b=hf['/buildings']

In [ ]:
del b['owner_units']

In [ ]:
hf['/buildings']=b

In [ ]:
af=hf['/zoning']

In [ ]:
af['max_far'] = af['max_far']/100.0

In [ ]:
af.head()

In [ ]:
hf['/buildings']=af

In [ ]:
af.head()

In [ ]:
hf['/zoning']=af

In [ ]:
hf.close()

In [ ]:
# for t in hf.keys():
#     df = hf[t]
#     print 
#     print t
#     print df.describe()
    
# building_types
# land_use_types

hf["building_types"]